Дараалал энгийнээр тайлбарлавал
1. Дата доторх өгүүлбэрийн үгсээ үндэс үг болгож өгүүлбэр хэлбэрт нь оруулаад датагаа янзалсан
2. Өгөгдсөн өгүүлбэрийн үндэс үгсийг олж алдаатай үгсийг зассан
3. Naive Bias алгоритм ашигласан MultinomialNB model ашиглаж сургалт хийсэн
Ашигласан дата агуулсан drive link:https://drive.google.com/drive/folders/1yJb2Bds2iwDiYswQMsxWUKEisRVuNtzj?usp=drive_link

#hunspell spellchecker install

In [14]:
!sudo apt-get update
!sudo apt-get install hunspell libhunspell-dev


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [15]:
!pip install hunspell

In [16]:
!pip install pyspellchecker

In [17]:
import pickle
import pandas as pd
import re
import os
import numpy as np
import gensim
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from gensim.models import Doc2Vec
import string
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import hunspell
from spellchecker import SpellChecker

In [21]:
train = pd.read_csv("train_data.csv", encoding='utf-8')

In [22]:
#78000 murtei 2attribut neg ni medee nuguu ni turultei data
# train = pd.read_csv("train_data.csv", encoding='utf-8', quoting=csv.QUOTE_NONE, on_bad_lines='skip', escapechar='\\')

NameError: name 'csv' is not defined

In [23]:
train.head()

,medee,turul
0,Киноны кадраас «Юрийн галавын үе» кино проката...,урлаг соёл
1,“Universal” кино компани Жадд Апатоугийн найру...,урлаг соёл
2,Баз Лурманы бүтээж байгаа “The Get Down” хөгжм...,урлаг соёл
3,“No wonder Films” болон “Алтан хальс” холбооны...,урлаг соёл
4,"Шударга өрсөлдөөн, хэрэглэгчийн төлөө газар 20...",урлаг соёл


In [24]:
hspell = hunspell.HunSpell('mn_MN.dic', 'mn_MN.aff') #mongol helnii ugsiin san
eng_spell = SpellChecker(language='en') #angli helnii ugsiin san

#aldaa undes ug

In [25]:
def ug_shalgah(test): # oruulsan uguulberiin ugsiin zuw buruug shalgah func
    ugnuud = test.replace(',', '').replace('.', '').replace('-','').split()

    zuw_ug = []
    buruu_ug = []

    for ug in ugnuud:
        if hspell.spell(ug) or ug in eng_spell:
            zuw_ug.append(ug)
        else:
            buruu_ug.append(ug)

    return zuw_ug, buruu_ug , ugnuud

def undes_ug(test): # ugsiin undes ugsiig oloh func
    stems = {}
    for ug in test:
        stem = hspell.stem(ug)
        if stem:
            stems[ug] = [s.decode('utf-8') for s in stem]
        else:
            stems[ug] = None

    return stems

def buruu_ug_sanal(buruu_ug): # buruu ugsed zuw ugs sanal bolgoh func
    suggestions = {}
    for ug in buruu_ug:
        suggestions[ug] = hspell.suggest(ug)
    return suggestions

#undes ugnuudiig uguulber bolgoh
def undes_ug_sentence(stems):
    sentence = []
    for word, stem_list in stems.items():
        if stem_list:
            sentence.append(stem_list[0])  # ehnii ug
        else:
            sentence.append(word)  # anhnii ug
    return " ".join(sentence)

test = """Австралийн  FlyQuest  баг  Perfect World Shanghai Major 2024  тэмцээнд оролцох эрхийн төлөө Хятадын  Lynn Vision Gaming -тай тоглож, 2:0 онооны харьцаатайгаар ялалт байгууллаа.  FlyQuest  баг өрсөлдөгчөө эхний  Nuke map  дээр 13:2, хоёр дахь  Vertigo map  дээр 13:8-аар хожлоо.  Ингэснээр erkaSt буюу Г.Эрдэнэцогт дасгалжуулагчийн хувиар анх удаа Counter Strike-2 (CS2) тоглоомын дэлхийн аварга шалгаруулах тэмцээнд оролцохоор болж байна. Тэрбээр тоглогчийн хувиар хоёр удаа MAJOR буюу дэлхийн аваргад оролцож, sticker гаргаж байсан.  Хятадын Шанхай хотод зохиогдох энэ удаагийн дэлхийн аваргад Ази-Номхон далайн бүсээс Австралийн  FlyQuest , Хятадын  Rare Atom , Монголын  The MongolZ  баг оролцохоор болж байна. 15:31 2024/11/13 Counter Strike-2 (CS2) тоглоомын энэ оны хоёр дахь Дэлхийн Аварга Шалгаруулах Тэмцээн (ДАШТ) буюу  Perfect World Shanghai Major 2024  ирэх сард Хятадын Шанхай хотод зохион байгуулагдана.  1.2 сая ам.долларын шагналын сантай тус тэмцээний урьдчилсан шатны тоглолтууд Шанхай хотод болж байна. Өчигдөр манай улсын  The MongolZ  баг  DRILLAS -ийг хожиж Ази-Номхон далайн бүсээс дэлхийн аваргад оролцох эрхээ авсан билээ.  Энэ бүсээс дэлхийн аваргад оролцох гурван эрхийн хоёрыг нь Хятадын  Rare Atom  болон  The MongolZ  баг дээд хүснэгтээс авсан. Гурав дахь эрхийн төлөө  DRILLAS , Австралийн  FlyQuest , Хятадын  Lynn Vision Gaming , Хонконгийн  TALON  багууд өрсөлдөж байгаа юм. Тэгвэл манай улсаас анх удаа дэлхийн аваргад оролцож,  sticker -ээ гаргасан тоглогч болох erkaSt буюу Г.Эрдэнэцогтын дасгалжуулж буй Австралийн  FlyQuest  баг  DRILLAS -ийн эсрэг өнөөдөр тоглож, 2:1-ээр ялалт байгууллаа. Австралийн багийн хувьд сүүлийн  Ancient map  дээр гурван удаа нэмэлт үе гаргасны эцэст 22:19-ээр хожив. Ийнхүү  FlyQuest  баг  DRILLAS -ийг тэмцээнээс хассанаар Хятадын  Lynn Vision Gaming , Хонконгийн  TALON  багийн аль хожсонтой нь дэлхийн аваргад оролцох эрхийн төлөө 19:00 цагаас тоглохоор болж байна. ErkaSt дасгалжуулагчийн хувиар гурав дахь удаагаа дэлхийн аваргын сонгон шалгаруулалтад оролцож байгаа юм. Хэрэв  FlyQuest  баг Ази-Номхон далайн бүсээс шалгарч чадвал Г.Эрдэнэцогт дасгалжуулагчийн хувиар анх удаа дэлхийн аваргад оролцоно.has context menu"""

zuw_ug, buruu_ug,ugnuud = ug_shalgah(test)

stems = undes_ug(ugnuud)

final_sentence = undes_ug_sentence(stems)

suggestions = buruu_ug_sanal(buruu_ug)

print("Зөв үгнүүд:", zuw_ug)
print("Буруу үгнүүд:", buruu_ug)
print("Үндэс үгнүүд: ", final_sentence)
print("\nСанал болгох үгс:")
for word, suggested in suggestions.items():
    print(f"{word}: {', '.join(suggested)}")


Зөв үгнүүд: ['Австралийн', 'баг', 'Perfect', 'World', 'Shanghai', 'Major', '2024', 'тэмцээнд', 'оролцох', 'эрхийн', 'төлөө', 'Хятадын', 'Vision', 'Gaming', 'тай', 'тоглож', 'онооны', 'харьцаатайгаар', 'ялалт', 'байгууллаа', 'баг', 'өрсөлдөгчөө', 'эхний', 'Nuke', 'map', 'дээр', 'хоёр', 'дахь', 'Vertigo', 'map', 'дээр', 'хожлоо', 'Ингэснээр', 'буюу', 'дасгалжуулагчийн', 'хувиар', 'анх', 'удаа', 'Counter', 'тоглоомын', 'дэлхийн', 'аварга', 'шалгаруулах', 'тэмцээнд', 'оролцохоор', 'болж', 'байна', 'Тэрбээр', 'тоглогчийн', 'хувиар', 'хоёр', 'удаа', 'MAJOR', 'буюу', 'дэлхийн', 'аваргад', 'оролцож', 'sticker', 'гаргаж', 'байсан', 'Хятадын', 'Шанхай', 'хотод', 'зохиогдох', 'энэ', 'удаагийн', 'дэлхийн', 'аваргад', 'далайн', 'бүсээс', 'Австралийн', 'Хятадын', 'Rare', 'Atom', 'Монголын', 'The', 'баг', 'оролцохоор', 'болж', 'байна', 'Counter', 'тоглоомын', 'энэ', 'оны', 'хоёр', 'дахь', 'Дэлхийн', 'Аварга', 'Шалгаруулах', 'Тэмцээн', 'буюу', 'Perfect', 'World', 'Shanghai', 'Major', '2024', 'ирэх', '

Жишээ өгүүлбэр: "суралцаад"

Hunspell энэ үгийг .dic файлд шууд хайна. Олдохгүй байж болно.

Hunspell дүрмийн кодын дагуу суралцаад үгийн залгаврыг хасаж, үндэс үг болох суралц гэж тодорхойлж, энэ нь .dic файлд байгаа эсэхийг дахин шалгана.

Хэрэв суралц үндэс толь бичигт байвал “суралцаад” үгийг зөв гэж тооцно.
Энэ зарчмаар Hunspell .aff болон .dic файлуудыг ашиглан зөв бичгийн шалгалт, үгийн үндэс үг гаргаж авах ажиллагааг гүйцэтгэдэг.

dic file д үндэс үгнүүд байрлаж байгаа
aff file д дүрэм байрлаж байгаа

суралц/F0 жишээн дээр суралц үг нь aff дэх F0 дүрэмд хамаарагдана

#uguulber tanilt

In [26]:
train = train[['medee', 'turul']]
train.head().T
train.groupby('turul').describe()

medee                                                            \
              count unique                                                top   
turul                                                                           
байгал орчин   5052   5040  Байгаль дэлхий дээр маш олон төрөл, зүйлийн ам...   
боловсрол      4472   4458  Монгол орон даяар өнөөдөр хичээлийн шинэ жилий...   
спорт         13431  13410  “Амжилтаа ахиулъя” төрийн бус байгууллагын сан...   
технологи      5525   5521  Япон эрдэмтэд Монголд тулгараад байгаа бэрхшээ...   
улс төр       13406  13406  Монгол Улсын Ерөнхий сайд Н.Алтанхуяг өнгөрсөн...   
урлаг соёл     7322   6968  Монгол Улсын нийслэл Улаанбаатар хот байгуулаг...   
хууль          8284   8279  Монгол улсын Ерөнхийлөгчөөс санаачлан боловсру...   
эдийн засаг   12391  12350                    Зураг дээр дарж томоор үзнэ үү.   
эрүүл мэнд     5778   4786  Зөвхөн танай гэр бүлд, ганцхан танд үйлчилдэг ...   

                   
             freq  
turul              
байгал орчин    2  
боловсрол       3  
спорт           4  
технологи       2  
улс төр         1  
урлаг соёл      5  
хууль           2  
эдийн засаг     7  
эрүүл мэнд      4

In [27]:
train.head()

,medee,turul
0,Киноны кадраас «Юрийн галавын үе» кино проката...,урлаг соёл
1,“Universal” кино компани Жадд Апатоугийн найру...,урлаг соёл
2,Баз Лурманы бүтээж байгаа “The Get Down” хөгжм...,урлаг соёл
3,“No wonder Films” болон “Алтан хальс” холбооны...,урлаг соёл
4,"Шударга өрсөлдөөн, хэрэглэгчийн төлөө газар 20...",урлаг соёл


In [28]:
train.turul.unique()

array(['урлаг соёл', 'эдийн засаг', 'эрүүл мэнд', 'хууль', 'улс төр',
       'спорт', 'технологи', 'боловсрол', 'байгал орчин'], dtype=object)

In [29]:
train['NUM_type_text']=train.turul.map({'байгал орчин':0,'боловсрол':1,'спорт':2,'технологи':3,'улс төр':4,'урлаг соёл':5,'хууль':6,'эдийн засаг':7,'эрүүл мэнд':8}) #turliin shoshguudaa toon helbert shiljuulj bga ni
train.head()
#ugsiig too bolgoh ngram_range ni 2 2 ugeer bagtsalj too bolgoj bga
vect = CountVectorizer(ngram_range=(2,2))
train = train.dropna()

x_train, x_test, y_train, y_test = train_test_split(train.medee, train.NUM_type_text, random_state=0, test_size=0.2)

#surgaltiin datag toon vectorluu horvuuleh
X_train = vect.fit_transform(x_train)
X_test = vect.transform(x_test)


Жишээ: Манай багш мундаг гэвэл "Манай багш" , "багш мундаг"

In [30]:
#surgalt
#Naive biasiin angilliin algorithm turul buguud dawtamjind suurilj text angildag model
mnb = MultinomialNB(alpha =0.2)
mnb.fit(X_train,y_train)
result= mnb.predict(X_test)
#model accuracy
print(accuracy_score(result,y_test))

0.8862089473336417


In [31]:
#garaltiin utga
def predict_news(news):
    test = vect.transform([news])
    pred= mnb.predict(test)
    if pred  == 0:
         return 'байгал орчин'
    elif pred ==1 :
        return 'боловсрол'
    elif pred ==2 :
        return 'спорт'
    elif pred ==3 :
        return 'технологи'
    elif pred ==4 :
        return 'улс төр'
    elif pred ==5 :
        return 'урлаг соёл'
    elif pred ==6 :
        return 'хууль'
    elif pred ==7 :
        return 'эдийн засаг'
    else :
        return 'эрүүл мэнд'

In [32]:
r = predict_news(final_sentence)
print (r)

спорт


#datagaa ynzalsan ni

In [33]:
# Үг шалгах функц
def ug_shalgah(test):
    ugnuud = test.replace(',', '').replace('.', '').split()

    zuw_ug = []
    buruu_ug = []

    for ug in ugnuud:
        if hspell.spell(ug) or ug in eng_spell:
            zuw_ug.append(ug)
        else:
            buruu_ug.append(ug)

    return zuw_ug, buruu_ug

# Үндэс үг олох функц
def undes_ug(zuw_ug):
    stems = {}
    for ug in zuw_ug:
        stem = hspell.stem(ug)
        if stem:
            stems[ug] = [s.decode('utf-8') for s in stem]
        else:
            stems[ug] = None

    return stems

# Буруу үгсэд санал болгох үгс
def buruu_ug_sanal(buruu_ug):
    suggestions = {}
    for ug in buruu_ug:
        suggestions[ug] = hspell.suggest(ug)
    return suggestions

# Медээний үндэс үг олох функц
def process_medee(medee_text):
    # Зөв болон буруу үгнүүдийг шалгах
    zuw_ug, buruu_ug = ug_shalgah(medee_text)
    # Зөв үгнүүдийн үндэс үгийг олох
    stems = undes_ug(zuw_ug)
    # Үндэс үгийг өгүүлбэр хэлбэрээр буцаах
    processed_text = " ".join([", ".join(stem_list) if stem_list else word for word, stem_list in stems.items()])
    return processed_text

# DataFrame-ийн medee баганыг боловсруулах
train['medee'] = train['medee'].apply(process_medee)

# Үр дүнг харах
train

,medee,turul,NUM_type_text
0,"кино кадр галав кино дараах эхэн амралт, амар ...",урлаг соёл,5
1,"кино компани найруул гэ хошин, хош шог хамтлаг...",урлаг соёл,5
2,"баз бүтээ бай Get хөгжим олон, ол ангит кино С...",урлаг соёл,5
3,wonder бол холбоо хамт бүтээл бол баримтат кин...,урлаг соёл,5
4,"шударга өрсөлдөөн хэрэглэ төлөө, төл газар 201...",урлаг соёл,5
...,...,...,...
75656,нийслэл эрүүл мэнд газар коронавирус халдвар с...,эрүүл мэнд,8
75657,нийслэл эрүүл мэнд газар бол харьяа эрүүл байг...,эрүүл мэнд,8
75658,"донор дэлхий жил бүрий, бүр зургадугаар сар 1 ...",эрүүл мэнд,8
75659,"нэгд арга хэмжээ, хэмж өнөөдөр цаг эрүүл яам б...",эрүүл мэнд,8


# a

In [34]:
from flask import Flask, render_template, request, jsonify
import hunspell
import os

app = Flask(__name__)

# Initialize Hunspell with dictionary files
def init_spellchecker():
    # Save dictionary files temporarily
    aff_content = """[Your mn_MN.aff content]"""
    dic_content = """[Your mn_MN.dic content]"""

    with open("mn_MN.aff", "w", encoding="utf-8") as f:
        f.write(aff_content)
    with open("mn_MN.dic", "w", encoding="utf-8") as f:
        f.write(dic_content)

    return hunspell.HunSpell("mn_MN.dic", "mn_MN.aff")

# Initialize spellchecker
spellchecker = init_spellchecker()

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/check', methods=['POST'])
def check():
    text = request.json.get('text', '')
    words = text.split()
    result = []

    for word in words:
        is_correct = spellchecker.spell(word)
        suggestions = spellchecker.suggest(word) if not is_correct else []
        result.append({
            'word': word,
            'correct': is_correct,
            'suggestions': suggestions
        })

    return jsonify(result)

# HTML template (save as templates/index.html)
"""
<!DOCTYPE html>
<html>
<head>
    <title>Mongolian Spell Checker</title>
    <style>
        .incorrect { color: red; }
        .suggestions { color: green; }
    </style>
</head>
<body>
    <h1>Mongolian Spell Checker</h1>
    <textarea id="text" rows="10" cols="50"></textarea>
    <br>
    <button onclick="checkSpelling()">Check Spelling</button>
    <div id="result"></div>

    <script>
        async function checkSpelling() {
            const text = document.getElementById('text').value;
            const response = await fetch('/check', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({text: text})
            });
            const data = await response.json();

            let html = '';
            data.forEach(item => {
                if (item.correct) {
                    html += `<span>${item.word}</span> `;
                } else {
                    html += `<span class="incorrect">${item.word}</span>`;
                    if (item.suggestions.length > 0) {
                        html += `<span class="suggestions">(${item.suggestions.join(', ')})</span> `;
                    }
                }
            });

            document.getElementById('result').innerHTML = html;
        }
    </script>
</body>
</html>
"""

if __name__ == '__main__':
    # For Google Colab, use:
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)
    app.run()

ModuleNotFoundError: No module named 'flask_ngrok'